In [1]:
import os
import sys
module_path = "../src"
#os.environ["WANDB_SILENT"] = "true" # Environment Variable to make wandb silent
if module_path not in sys.path:
    sys.path.append(module_path)
# 1. General Libraries
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# 2. DataModule & Class Libraries
from utils.label_encoder import label_encoder_target
from utils.calc_stat import calc_dataset_mean_std
from utils.FeatureExtractor import get_vector_representation,prepare_projection
from dataset.ImageDataModule import ImageDataModule
from dataset.ImageDataset import ImageDataset
from models.EfficientNetClass import EfficientNetClass
from models.ResNetClass import ResNetClass
# 3. Pytorch & Pytorch Lightning Libraries
from pytorch_lightning import Trainer,seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import torch
# 4. Wandb Tracker Experiements
import wandb

In [5]:
DATA_PATH = '../data/molecular_dataset.csv'
BASE_PATH = Path('/mnt')
BASE_IMG_DIR = BASE_PATH /'subfigure-classification'
SEED = 42
model_path = '/mnt/artifacts/models/Molecular/molecular_v1.pt'

In [6]:
df = pd.read_csv(DATA_PATH, sep='\t')
le_encoder,dict_label = label_encoder_target(df,target_col='higher_modality')
df.head()

,img,modality,set,source,img_path,higher_modality,caption,split_set
0,1423-0127-16-109-3-1.jpg,D3DR,TRAIN,clef16,2016/train/D3DR/1423-0127-16-109-3-1.jpg,MOLECULAR,A. Three dimensional model of class C protein ...,VAL
1,1423-0127-16-109-3-2.jpg,D3DR,TRAIN,clef16,2016/train/D3DR/1423-0127-16-109-3-2.jpg,MOLECULAR,A. Three dimensional model of class C protein ...,TRAIN
2,1423-0127-17-34-3-1.jpg,D3DR,TRAIN,clef16,2016/train/D3DR/1423-0127-17-34-3-1.jpg,MOLECULAR,Three domains of AAG A (or C)-chain: (a) domai...,TRAIN
3,1423-0127-17-34-3-2.jpg,D3DR,TRAIN,clef16,2016/train/D3DR/1423-0127-17-34-3-2.jpg,MOLECULAR,Three domains of AAG A (or C)-chain: (a) domai...,VAL
4,1423-0127-17-34-3-3.jpg,D3DR,TRAIN,clef16,2016/train/D3DR/1423-0127-17-34-3-3.jpg,MOLECULAR,Three domains of AAG A (or C)-chain: (a) domai...,TRAIN


In [8]:
# 3.1 Load the Resnet Model
model = ResNetClass.load_from_checkpoint(model_path)
print('*'*5,' Resnet Model Parameters ','*'*5)
print(model.hparams)
print('*'*5,' Resnet Model Children ','*'*5)
children_counter = 0
for n,c in model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1

*****  Resnet Model Parameters  *****
"class_weights":   [0.98866213 1.5970696  0.73400673]
"fine_tuned_from": whole
"lr":              1e-05
"mean_dataset":    [0.851899  0.8511532 0.8413842]
"metric_monitor":  val_avg_loss
"mode_scheduler":  min
"name":            resnet101
"num_classes":     3
"pretrained":      True
"std_dataset":     [0.2773475  0.27720553 0.28995028]
*****  Resnet Model Children  *****
Children Counter:  0  Layer Name:  model


In [9]:
prepare_projection(model ,le_encoder,DATA_PATH,BASE_IMG_DIR,SEED,CLASSF ='molecular' ,VERSION = 1)

Feature Vector for training: 



Feature Vector for Validation: 



Feature Vector for Test: 



***** PCA *****
***** UMAP *****
